# FAF5 Network Analysis with AequilibraE

This notebook demonstrates how to:
1. Load the FAF5 network from Geodatabase format
2. Create an AequilibraE project and import the network
3. Calculate shortest travel times/costs between OD-pairs
4. Compute shortest paths between OD-pairs
5. Generate route choice sets between OD pairs
6. Perform traffic assignment using path-sized logit method


In [ ]:
# Imports
import os
import geopandas as gpd
import pandas as pd
import numpy as np
from uuid import uuid4
from tempfile import gettempdir
from os.path import join
from shapely.geometry import Point, LineString

from aequilibrae import Project
from aequilibrae.paths import RouteChoice
from aequilibrae.paths.traffic_assignment import TrafficAssignment
from aequilibrae.paths.traffic_class import TrafficClass
from aequilibrae.matrix import AequilibraeMatrix

print("Libraries imported successfully")


## 1. Load FAF5 Network Data

Load the FAF5 network layers from the Geodatabase format.


In [ ]:
# Path to the FAF5 Geodatabase
gdb_path = "Networks/Geodatabase Format/FAF5Network.gdb"

# Load FAF5 layers
print("Loading FAF5 network layers...")
links_gdf = gpd.read_file(gdb_path, layer="FAF5_Links")
nodes_gdf = gpd.read_file(gdb_path, layer="FAF5_Nodes")

print(f"Loaded {len(links_gdf)} links and {len(nodes_gdf)} nodes")
print(f"\nLinks columns: {list(links_gdf.columns)}")
print(f"\nNodes columns: {list(nodes_gdf.columns)}")
print(f"\nLinks head:")
links_gdf.head()


In [ ]:
# Display nodes head
print("Nodes head:")
nodes_gdf.head()


## 2. Create AequilibraE Project

Create a new AequilibraE project and import the FAF5 network data.


In [ ]:
# Create a new AequilibraE project
project_folder = join(gettempdir(), f"faf5_analysis_{uuid4().hex[:8]}")
project = Project()
project.new(project_folder)
print(f"Created project at: {project_folder}")


In [ ]:
# Prepare nodes first - we need to identify centroids and extract node coordinates
nodes = project.network.nodes

# Process nodes
print("Adding nodes to project...")
node_coords = {}  # Store node coordinates for link creation

for idx, row in nodes_gdf.iterrows():
    node = nodes.new()
    node.node_id = int(row['ID'])
    node.is_centroid = 1 if row.get('Centroid', 0) == 1 else 0
    
    # Store geometry/coordinates
    if row.geometry is not None:
        if hasattr(row.geometry, 'x') and hasattr(row.geometry, 'y'):
            node.geometry = Point(row.geometry.x, row.geometry.y)
            node_coords[node.node_id] = (row.geometry.x, row.geometry.y)
        else:
            node.geometry = row.geometry
            # Extract coordinates from geometry if it's a Point
            if row.geometry.geom_type == 'Point':
                node_coords[node.node_id] = (row.geometry.x, row.geometry.y)
    
    node.save()

print(f"Added {len(nodes_gdf)} nodes")


In [ ]:
# Add required fields to links table if needed
links = project.network.links
link_fields = links.fields

# Add fields for FAF5 data
try:
    link_fields.add("travel_time", "Free flow travel time", "REAL")
    link_fields.add("speed", "Speed limit", "REAL")
    link_fields.add("source_id", "Original FAF5 link ID", "INTEGER")
    links.refresh_fields()
except:
    pass  # Fields may already exist

print("Link fields prepared")


In [ ]:
# Process links
# FAF5 links have DIR field: 0 = bidirectional, 1 = AB only
# We need to extract a_node and b_node from geometry endpoints

print("Adding links to project...")
link_count = 0

# Create spatial index for faster node matching
from shapely.strtree import STRtree
node_points = [Point(geom.x, geom.y) if geom is not None and geom.geom_type == 'Point' else geom 
               for geom in nodes_gdf.geometry]
node_tree = STRtree(node_points)
node_ids = nodes_gdf['ID'].values

# Helper function to find nearest node
def find_nearest_node(point, max_distance=0.01):
    """Find nearest node to a point within max_distance (degrees)"""
    if point is None:
        return None
    # Query spatial index
    candidates = node_tree.query(point.buffer(max_distance))
    if len(candidates) == 0:
        return None
    # Find closest
    distances = [point.distance(node_points[i]) for i in candidates]
    nearest_idx = candidates[np.argmin(distances)]
    return int(node_ids[nearest_idx])

for idx, row in links_gdf.iterrows():
    link_id = int(row['ID'])
    direction = int(row.get('DIR', 0))  # 0 = bidirectional, 1 = AB only
    
    # Get geometry
    geom = row.geometry
    
    # Extract endpoints from geometry
    if geom is not None and geom.geom_type in ['LineString', 'MultiLineString']:
        # For LineString, get first and last point
        if geom.geom_type == 'LineString':
            coords = list(geom.coords)
            start_point = Point(coords[0])
            end_point = Point(coords[-1])
        else:  # MultiLineString
            # Use first and last point of the entire geometry
            first_line = geom.geoms[0]
            last_line = geom.geoms[-1]
            start_point = Point(first_line.coords[0])
            end_point = Point(last_line.coords[-1])
        
        # Find nearest nodes to endpoints
        a_node_id = find_nearest_node(start_point)
        b_node_id = find_nearest_node(end_point)
        
        if a_node_id is None or b_node_id is None:
            continue
        
        # Get travel time based on direction
        travel_time_ab = row.get('AB_FreeFlowTime', np.nan)
        travel_time_ba = row.get('BA_FreeFlowTime', np.nan)
        length = row.get('LENGTH', 0.0)
        
        # Get capacity (use a default if not available)
        capacity_ab = row.get('AB_Lanes', 1.0) * 1000 if not np.isnan(row.get('AB_Lanes', np.nan)) else 1000
        capacity_ba = row.get('BA_Lanes', 1.0) * 1000 if not np.isnan(row.get('BA_Lanes', np.nan)) else 1000
        
        # Create AB link (direction = 1)
        if not np.isnan(travel_time_ab) and travel_time_ab > 0:
            link_ab = links.new()
            link_ab.link_id = link_id if direction == 1 else link_id * 1000 + 1
            link_ab.a_node = a_node_id
            link_ab.b_node = b_node_id
            link_ab.direction = 1
            link_ab.distance = length if not np.isnan(length) else 0.0
            link_ab.travel_time = travel_time_ab
            link_ab.capacity = capacity_ab
            link_ab.speed = row.get('AB_FinalSpeed', np.nan) if not np.isnan(row.get('AB_FinalSpeed', np.nan)) else None
            link_ab.source_id = link_id
            link_ab.modes = 'c'  # Car mode
            link_ab.geometry = geom
            link_ab.save()
            link_count += 1
        
        # Create BA link if bidirectional (direction = 0)
        if direction == 0 and not np.isnan(travel_time_ba) and travel_time_ba > 0:
            link_ba = links.new()
            link_ba.link_id = link_id * 1000 + 2
            link_ba.a_node = b_node_id
            link_ba.b_node = a_node_id
            link_ba.direction = 1
            link_ba.distance = length if not np.isnan(length) else 0.0
            link_ba.travel_time = travel_time_ba
            link_ba.capacity = capacity_ba
            link_ba.speed = row.get('BA_FinalSpeed', np.nan) if not np.isnan(row.get('BA_FinalSpeed', np.nan)) else None
            link_ba.source_id = link_id
            link_ba.modes = 'c'  # Car mode
            link_ba.geometry = geom
            link_ba.save()
            link_count += 1

print(f"Added {link_count} links")


## 3. Build Graphs

Build the graph structures needed for path computation and traffic assignment.


In [ ]:
# Build graphs
print("Building graphs...")
project.network.build_graphs()

# Get the graph for cars
graph = project.network.graphs["c"]
print(f"Available graphs: {list(project.network.graphs.keys())}")

# Set graph cost field to travel_time
graph.set_graph("travel_time")

# Set skimming fields
graph.set_skimming(["travel_time", "distance"])

# Get centroids
centroids = project.network.nodes.data[project.network.nodes.data.is_centroid == 1]
centroid_ids = centroids.node_id.values
print(f"Found {len(centroid_ids)} centroids")

# Prepare graph with centroids
graph.prepare_graph(centroid_ids)
print("Graph prepared successfully")


## 4. Calculate Shortest Travel Times/Costs

Calculate shortest travel times between random OD-pairs.


In [ ]:
# Select random OD-pairs from centroids
np.random.seed(42)  # For reproducibility
n_pairs = 10
random_indices = np.random.choice(len(centroid_ids), size=min(n_pairs * 2, len(centroid_ids)), replace=False)
od_pairs = [(centroid_ids[random_indices[i]], centroid_ids[random_indices[i+1]]) 
            for i in range(0, len(random_indices)-1, 2)][:n_pairs]

print(f"Selected {len(od_pairs)} OD-pairs for analysis")
print("OD-pairs:", od_pairs[:5], "...")


In [ ]:
# Calculate shortest path costs for each OD-pair
results = []
links_data = project.network.links.data.set_index('link_id')

for origin, destination in od_pairs:
    try:
        res = graph.compute_path(origin, destination)
        if res is not None and res.path is not None and len(res.path) > 0:
            # Get travel time from milepost (cumulative cost)
            travel_time = res.milepost[-1] if len(res.milepost) > 0 else np.nan
            
            # Calculate distance by summing link distances along the path
            path_links = res.path
            distance = 0.0
            for link_id in path_links:
                link_id_abs = abs(link_id)
                if link_id_abs in links_data.index:
                    link_dist = links_data.loc[link_id_abs, 'distance']
                    if pd.notna(link_dist):
                        distance += link_dist
            
            results.append({
                'origin': origin,
                'destination': destination,
                'travel_time': travel_time,
                'distance': distance,
                'path_length': len(res.path)
            })
    except Exception as e:
        print(f"Error computing path from {origin} to {destination}: {e}")
        continue

# Create results dataframe
shortest_costs_df = pd.DataFrame(results)
print(f"\nComputed shortest paths for {len(shortest_costs_df)} OD-pairs")
shortest_costs_df.head(10)


## 5. Compute Shortest Paths

Compute detailed shortest paths between OD-pairs and extract path information.


In [ ]:
# Compute detailed shortest paths
path_results = []
links_data = project.network.links.data.set_index('link_id')

# Select a few OD-pairs for detailed path analysis
selected_od_pairs = od_pairs[:5]

for origin, destination in selected_od_pairs:
    try:
        res = graph.compute_path(origin, destination)
        if res is not None and res.path is not None and len(res.path) > 0:
            # Get travel time from milepost
            travel_time = res.milepost[-1] if len(res.milepost) > 0 else np.nan
            
            # Calculate distance by summing link distances along the path
            path_links = res.path
            distance = 0.0
            for link_id in path_links:
                link_id_abs = abs(link_id)
                if link_id_abs in links_data.index:
                    link_dist = links_data.loc[link_id_abs, 'distance']
                    if pd.notna(link_dist):
                        distance += link_dist
            
            path_info = {
                'origin': origin,
                'destination': destination,
                'path_nodes': res.path_nodes.tolist() if hasattr(res, 'path_nodes') else [],
                'path_links': res.path.tolist(),
                'travel_time': travel_time,
                'distance': distance,
                'num_nodes': len(res.path_nodes) if hasattr(res, 'path_nodes') else 0,
                'num_links': len(res.path)
            }
            path_results.append(path_info)
            print(f"Path from {origin} to {destination}: {path_info['num_links']} links, "
                  f"travel time: {path_info['travel_time']:.2f}, distance: {path_info['distance']:.2f}")
    except Exception as e:
        print(f"Error computing path from {origin} to {destination}: {e}")
        continue

paths_df = pd.DataFrame(path_results)
print(f"\nComputed detailed paths for {len(paths_df)} OD-pairs")
paths_df[['origin', 'destination', 'num_links', 'travel_time', 'distance']]


## 6. Generate Route Choice Sets

Generate route choice sets between OD pairs using the RouteChoice class.


In [ ]:
# Initialize RouteChoice
print("Initializing RouteChoice...")
rc = RouteChoice(graph)

# Set choice set generation parameters
# Using BFS-LE (Breadth-First Search with Link Elimination) algorithm
rc.set_choice_set_generation("bfsle", max_routes=5, penalty=1.05)

# Select a smaller subset of OD-pairs for route choice (this can be computationally intensive)
route_choice_od_pairs = selected_od_pairs[:3]
print(f"Generating route choice sets for {len(route_choice_od_pairs)} OD-pairs...")

# Prepare and execute route choice
rc.prepare(route_choice_od_pairs)
rc.execute(perform_assignment=False)

# Get results
choice_set_results = rc.get_results()
print(f"\nGenerated route choice sets:")
print(choice_set_results.head())


In [ ]:
# Display route choice set details
if choice_set_results is not None and len(choice_set_results) > 0:
    print("Route Choice Set Summary:")
    print(f"Total OD-pairs: {len(choice_set_results)}")
    
    # Show number of routes per OD-pair
    if 'route set' in choice_set_results.columns:
        route_counts = choice_set_results.apply(lambda x: len(x['route set']) if isinstance(x['route set'], (list, tuple)) else 0, axis=1)
        print(f"\nRoutes per OD-pair:")
        print(route_counts.describe())
    
    # Display first few results
    print("\nFirst few route choice sets:")
    display_cols = [col for col in choice_set_results.columns if col != 'route set']
    print(choice_set_results[display_cols].head())
else:
    print("No route choice sets generated")


## 7. Traffic Assignment with Path-Sized Logit

Perform traffic assignment using the path-sized logit method.


In [ ]:
# Create a demand matrix for traffic assignment
# For demonstration, we'll create a simple random demand matrix
# In practice, you would load actual OD demand data

print("Creating demand matrix...")

# Get number of zones (centroids)
num_zones = len(centroid_ids)
zone_index = centroid_ids

# Create a simple demand matrix (random values for demonstration)
# In practice, load actual demand data
np.random.seed(42)
demand_matrix = np.random.rand(num_zones, num_zones) * 1000
# Set diagonal to zero (no internal trips)
np.fill_diagonal(demand_matrix, 0)

# Create AequilibraeMatrix
matrix_file = join(project_folder, "demand.aem")
aem = AequilibraeMatrix()
kwargs = {
    'file_name': matrix_file,
    'zones': num_zones,
    'matrix_names': ['demand']
}
aem.create_empty(**kwargs)
aem.matrix['demand'][:, :] = demand_matrix[:, :]
aem.index[:] = zone_index[:]

# Set computational view
aem.computational_view(['demand'])
print(f"Created demand matrix with {num_zones} zones")
print(f"Total demand: {demand_matrix.sum():.0f} trips")


In [ ]:
# Set up traffic assignment with path-sized logit using RouteChoice
print("Setting up route choice assignment with path-sized logit...")

# Create RouteChoice object
rc_assignment = RouteChoice(graph)

# Add demand matrix to route choice
rc_assignment.add_demand(aem)

# Set choice set generation parameters
rc_assignment.set_choice_set_generation("bfsle", max_routes=5, penalty=1.05)

# Set path-sized logit parameters
rc_assignment.set_path_size_logit(beta_psl=1.0, min_share=0.01)

# Get OD pairs from demand matrix for preparation
od_list = []
for i, orig in enumerate(centroid_ids):
    for j, dest in enumerate(centroid_ids):
        if demand_matrix[i, j] > 0:
            od_list.append((int(orig), int(dest)))

print(f"Prepared route choice for {len(od_list)} OD pairs with demand")
print("Route choice configured with path-sized logit")


In [ ]:
# Execute route choice assignment with path-sized logit
print("Executing route choice assignment with path-sized logit...")
print("This may take several minutes depending on network size and number of OD pairs...")

try:
    # Prepare route choice with OD pairs
    rc_assignment.prepare(od_list)
    
    # Execute route choice with assignment (path-sized logit)
    rc_assignment.execute(perform_assignment=True)
    
    print("\nRoute choice assignment completed successfully!")
    
    # Get results
    choice_results = rc_assignment.get_results()
    
    print("\nRoute Choice Assignment Results Summary:")
    print(f"Total OD-pairs processed: {len(choice_results) if choice_results is not None else 0}")
    
    # Get link loads from route choice
    if hasattr(rc_assignment, 'link_loads'):
        link_volumes = rc_assignment.link_loads
        print(f"\nLink volumes computed for {len(link_volumes)} links")
        print(f"Total assigned volume: {link_volumes.sum():.0f}")
        print(f"Average link volume: {link_volumes.mean():.2f}")
        print(f"Maximum link volume: {link_volumes.max():.2f}")
    else:
        print("\nNote: Link volumes are stored in the route choice results")
        print("Access via rc_assignment.get_results() for detailed route-level assignments")
    
except Exception as e:
    print(f"Error during route choice assignment: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Display link volumes on network
if 'link_volumes' in locals() and link_volumes is not None:
    # Get link data
    links_data = project.network.links.data
    
    # Add volumes to links dataframe
    links_with_volumes = links_data.copy()
    links_with_volumes['volume'] = 0.0
    
    # Map volumes to links (this is a simplified mapping)
    # In practice, you would use the proper link_id mapping from assignment results
    print("Link volumes summary:")
    print(f"Links with volume > 0: {(link_volumes > 0).sum()}")
    print(f"Top 10 link volumes:")
    
    # Get top volumes
    if len(link_volumes) > 0:
        top_volumes = pd.Series(link_volumes).nlargest(10)
        print(top_volumes)


## Summary

This notebook demonstrated:
1. ✅ Loading FAF5 network from Geodatabase format
2. ✅ Creating AequilibraE project and importing network
3. ✅ Calculating shortest travel times/costs between OD-pairs
4. ✅ Computing shortest paths with detailed information
5. ✅ Generating route choice sets using BFS-LE algorithm
6. ✅ Performing traffic assignment with path-sized logit method

The analysis is complete. You can now explore the results further or modify parameters for different scenarios.


In [ ]:
# Clean up - close the project
project.close()
print("Project closed")
